In [1]:
%matplotlib inline

import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

### Reference for API of tensorflow
 - https://www.tensorflow.org/api_docs/

### 0.tensorflow 코드 기본 형태
  - 준비 작업 
      - 데이터 전처리
      - 네트워크 정의
  - 실제 작업 수행 (in session)
      - session 실행
      - session.run(output, input_values)

In [ ]:
"""
    Construct model/network
    TODO: preparing training data and construct model
"""
# prepare training data
x_train = [1,2,3,4]     # input data
y_train = [0,-1,-2,-3]  # label

# Model parameters
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
# Model input and output
x = tf.placeholder(tf.float32)
linear_model = W * x + b
y = tf.placeholder(tf.float32)
# loss
loss = tf.reduce_sum(tf.square(linear_model - y)) # sum of the squares
# optimizer
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

# initialize variables
init = tf.global_variables_initializer()

"""
    Running graph in GPU
    TODO: running and updating model
"""
with tf.Session() as sess:    
    # session initialization
    sess.run(init)
    
    curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
    print("Initial W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))
    
    # training the model 
    for i in range(1000):
        sess.run(train, {x:x_train, y:y_train})
    
    # evaluating model
    curr_W, curr_b, curr_loss  = sess.run([W, b, loss], {x:x_train, y:y_train})
    print("Final W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

### 1. Tensor 종류 
  - constant, placeholder, variable
  
#### 1.1 Constant (상수)
  - 선언과 함께 초기화됨
  - 값이 바뀌지 않음

In [2]:
# constant (상수): no input, fixed (or predefined) output
node1 = tf.constant(3.0, tf.float32)
node2 = tf.constant(4.0) # also tf.float32 implicitly
print(node1)
print(node2)

Tensor("Const:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(), dtype=float32)


In [3]:
# check what is different
with tf.Session() as sess:
    print(sess.run([node1, node2]))

[3.0, 4.0]


#### 1.2 Placeholder
  - 유저가 제공하는 값으로 데이터에 주로 사용 (e.g. 이미지, 텍스트 등)

In [4]:
node1 = tf.placeholder(tf.float32)
node2 = tf.placeholder(tf.float32)
print(node1)
print(node2)
print("")

with tf.Session() as sess:
    print("one input for node1:")
    print(sess.run(node1, {node1:7}))
    print(sess.run(node1, {node1:3}))
    print("two inputs for node1 and node2:")
    print(sess.run([node1,node2], {node1: [3,5], node2: [15,25]}))
    print("--------------------------------------\n")
    
    output = sess.run([node1,node2], {node1: [3,5], node2: [15,25]})
    print("First output: ", output[0])
    print("Second output: ", output[1])

Tensor("Placeholder:0", dtype=float32)
Tensor("Placeholder_1:0", dtype=float32)

one input for node1:
7.0
3.0
two inputs for node1 and node2:
[array([ 3.,  5.], dtype=float32), array([ 15.,  25.], dtype=float32)]
--------------------------------------

First output:  [ 3.  5.]
Second output:  [ 15.  25.]


#### 1.3 Variable
  - 모델/네트워크의 파라미터에 주로 사용
  - 선언과 함께 초기화되지 않기 때문에 tf.global_variables_initializer()를 호출해줘야 함

In [5]:
# 모델 정의 (변수/인풋 선언)
W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
linear_model = W * x + b # x * 0.3 - 0.3
update_b = tf.assign(b, [-.6])  # 변수 값 변환

# 변수 (W, b) 초기화
init = tf.global_variables_initializer()

with tf.Session() as sess:
    # session에서 변수 초기화
    sess.run(init)
    
    # model 실행
    print(sess.run(linear_model, {x:[1,2,3,4]}))
    
    # change variables
    print("\nAfter chaning bias from -0.3 to -0.6") # linear_model = x * 0.3 - 0.6
    print(sess.run(update_b))
    print(sess.run(linear_model, {x:[1,2,3,4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


### 2. Mathematic operation
 - reference:<br>
 https://www.tensorflow.org/versions/master/api_guides/python/math_ops#Arithmetic_Operators
 - 주요 API (기본 연산) <br>
     - tf.add(x,y): element-wise addition
     - tf.subtract(x,y): element-wise subtraction
     - tf.multiply(x,y): element-wise multiplication
     - tf.scalar_mul(scalar, x): multipy a scalar to x
     - tf.div(x,y): element-wise division 
     - tf.mod(x,y): element-wise remainder of division
 - 주요 API (산술 연산): <br>
     - tf.abs(), tf.neg(), tf.inv(), tf.square(), tf.sqrt(), tf.pow(),tf.log(), tf.ceil(), tf.floor(), tf.maximum(), tf.minimum(), tf.cos(), tf.sin(), tf.tan()
 - 주요 API (벡터/매트릭스 연산): <br>
     - tf.matmul(), tf.reduce_mean(), tf.reduce_sum(), reduce_min(), reduce_max(), 
 - 필요에 따라 API를 검색해서 사용해주세요.

#### 2.1 기본 연산

In [6]:
node1 = tf.constant(0.3)
node2 = tf.constant(0.6)

# element-wise addition (덧셈)
add_node3_1 = tf.add(node1, node2)
add_node3_2 = node1 + node2

# element-wise multiplication (곱셈)
mul_node4_1 = tf.multiply(node1, node2)
mul_node4_2 = node1 * node2

print("add_node3_1: ", add_node3_1)
print("add_node3_2: ", add_node3_2)
print("mul_node4_1: ", mul_node4_1)
print("mul_node4_2: ", mul_node4_2)
print("")
with tf.Session() as sess:
    print("sess.run(add_node3_1): ", sess.run(add_node3_1))
    print("sess.run(add_node3_2): ", sess.run(add_node3_2))
    print("sess.run(mul_node4_1): ", sess.run(mul_node4_1))
    print("sess.run(mul_node4_2): ", sess.run(mul_node4_2))

add_node3_1:  Tensor("Add:0", shape=(), dtype=float32)
add_node3_2:  Tensor("add_1:0", shape=(), dtype=float32)
mul_node4_1:  Tensor("Mul:0", shape=(), dtype=float32)
mul_node4_2:  Tensor("mul_1:0", shape=(), dtype=float32)

sess.run(add_node3_1):  0.9
sess.run(add_node3_2):  0.9
sess.run(mul_node4_1):  0.18
sess.run(mul_node4_2):  0.18


#### 2.2 산술 연산

In [8]:
inp = np.arange(0,4).reshape(2,2)
node1 = tf.constant(inp, tf.float32)
print("input matrix:")
print(inp, "\n")

# Computing suare values
square_node2 = tf.square(node1)

with tf.Session() as sess:
    print("sess.run(square_node2): \n", sess.run(square_node2))

input matrix:
[[0 1]
 [2 3]] 

sess.run(square_node2): 
 [[ 0.  1.]
 [ 4.  9.]]


#### 2.2 벡터/매트릭스 연산

In [9]:
inp = np.random.randn(2,2)
node1 = tf.constant(inp, tf.float32)
print("input matrix:")
print(inp, "\n")

# finding maximum value
max_node2 = tf.reduce_max(node1)
max_node2_1dim = tf.reduce_max(node1, axis=0)
max_node2_2dim = tf.reduce_max(node1, axis=1)

with tf.Session() as sess:
    print("sess.run(max_node2): ", sess.run(max_node2))
    print("sess.run(max_node2_1dim): ", sess.run(max_node2_1dim))
    print("sess.run(max_node2_2dim): ", sess.run(max_node2_2dim))

input matrix:
[[-0.71693953  3.77481167]
 [ 0.27593423  2.25580511]] 

sess.run(max_node2):  3.77481
sess.run(max_node2_1dim):  [ 0.27593422  3.77481174]
sess.run(max_node2_2dim):  [ 3.77481174  2.25580502]


### 3. Model Training
#### 3.1 Loss 정의
 - cross-entropy loss, l2 normalization loss, hinge loss, etc

In [14]:
# define ground-truth label
x = tf.placeholder(tf.float32)
true_label = tf.placeholder(tf.float32)

W = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
linear_model = W * x + b # x * 0.3 - 0.3

# define regression loss
squared_diff = tf.square(linear_model - true_label)
loss = tf.reduce_sum(squared_diff)

#### 3.2 모델 업데이트
 - loss에 대한 파라미터의 gradient 계산
 - gradient를 이용하여 모델 파라미터 업데이트
     - $\theta=\theta-learning\_rate*d\theta$
 - 위의 작업은 optimizer를 이용하여 간단하게 구현 가능
 - optimizer 종류
      - GradientDescentOptimizer, AdagradOptimizer, RMSPropOptimizer, AdamOptimizer

In [24]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    curr_W, curr_b, curr_loss, _ = \
            sess.run([W,b,loss,train], {x:[1,2,3,4], true_label:[0,-1,-2,-3]})
    print("W: %s b: %s loss: %s"%(curr_W, curr_b, curr_loss))

W: [ 0.30000001] b: [-0.30000001] loss: 23.66
